In [1]:
import pandas as pd
import numpy as np
import os


search_dir = "/home/n7487/Downloads/Geo/data/"
os.chdir(search_dir)
files = filter(os.path.isfile, os.listdir(search_dir))
files = [os.path.join(search_dir, f) for f in files]
files = [x for x in files if x.endswith('xlsx')]


df_kolkata = []
df_lucknow = []
count_kolkata = 0
count_lucknow = 0
start_index = 16

## add_PM_values(df,df1)
### There are a lot of 'None' values in the data collated from CPCB. 
### To Calculate the averages, I'm mainting a counter which gets reduced by 1 each time a 'None' is detected in the dataframes. I'm also resetting the 'None' PM2.5 value to be 0.
### Finally adding the PM2.5 values and the counter to later find accurate averages.

In [2]:
def add_PM_values(df,df1):

    if 'None' in df['PM2.5'].values:
        df['PM2.5']= df['PM2.5'].astype(str)
        df.loc[df['PM2.5'].str.lower().isin(['none']), 'count'] -= 1             
        df.loc[df['PM2.5'] == 'None', 'PM2.5'] = 0
        df['PM2.5']= df['PM2.5'].astype(float64)
        
    if 'None' in df1['PM2.5'].values:
        df1['PM2.5']= df1['PM2.5'].astype(str)
        df1.loc[df1['PM2.5'].str.lower().isin(['none']), 'count'] -= 1            
        df1.loc[df1['PM2.5'] == 'None', 'PM2.5'] = 0
        df1['PM2.5']= df1['PM2.5'].astype(np.float64)
    df['PM2.5'] = df['PM2.5'] + df1['PM2.5']
    df['count'] = df['count'] + df1['count']
    return df

### Collecting data for all stations in Lucknow and Kolata


In [3]:
for f in files:
    df_temp = pd.read_excel(f)
    df_temp.columns = ['From', 'To', 'PM2.5']
    df_temp['count'] = 1
    if df_temp.iloc[4]['To'] == 'Lucknow':                # df_temp.iloc[4]['To'] is the city name
        if len(df_lucknow)==0:
            df_lucknow = df_temp[start_index:].reset_index(drop=True)
        else:
            df_lucknow = add_PM_values(df_lucknow, df_temp[start_index:].reset_index(drop=True))
    else:
        if len(df_kolkata)==0:
            df_kolkata = df_temp[start_index:].reset_index(drop=True)
        else:
            df_kolkata = add_PM_values(df_kolkata, df_temp[start_index:].reset_index(drop=True))

### Calculating each day's PM2.5 average across all stations for both cities using the initial counter consisting of non-zero values

In [4]:
df_lucknow['PM2.5_StationAverage'] = df_lucknow['PM2.5'] /df_lucknow['count'] 
df_kolkata['PM2.5_StationAverage'] = df_kolkata['PM2.5'] /df_kolkata['count'] 

### Output:

In [5]:
df_lucknow

,From,To,PM2.5,count,PM2.5_StationAverage
0,01-01-2020 00:00,02-01-2020 00:00,1094.75,4,273.6875
1,02-01-2020 00:00,03-01-2020 00:00,709.23,4,177.3075
2,03-01-2020 00:00,04-01-2020 00:00,474.5,4,118.625
3,04-01-2020 00:00,05-01-2020 00:00,418.49,4,104.6225
4,05-01-2020 00:00,06-01-2020 00:00,396.39,4,99.0975
...,...,...,...,...,...
903,22-06-2022 00:00,23-06-2022 00:00,213.67,5,42.734
904,23-06-2022 00:00,24-06-2022 00:00,258.45,5,51.69
905,24-06-2022 00:00,25-06-2022 00:00,275.22,5,55.044
906,25-06-2022 00:00,26-06-2022 00:00,261.87,5,52.374


In [6]:
df_kolkata

,From,To,PM2.5,count,PM2.5_StationAverage
0,01-01-2020 00:00,02-01-2020 00:00,630.43,5,126.086
1,02-01-2020 00:00,03-01-2020 00:00,675.82,6,112.636667
2,03-01-2020 00:00,04-01-2020 00:00,298.24,6,49.706667
3,04-01-2020 00:00,05-01-2020 00:00,306.18,6,51.03
4,05-01-2020 00:00,06-01-2020 00:00,541.81,6,90.301667
...,...,...,...,...,...
903,22-06-2022 00:00,23-06-2022 00:00,114.86,6,19.143333
904,23-06-2022 00:00,24-06-2022 00:00,92.87,6,15.478333
905,24-06-2022 00:00,25-06-2022 00:00,106.71,6,17.785
906,25-06-2022 00:00,26-06-2022 00:00,113.51,6,18.918333


In [7]:
df_lucknow.to_csv('./df_lucknow.csv',index=None)
df_kolkata.to_csv('./df_kolkata.csv',index=None)


## Calculate Moving Averages

In [8]:

window = 365
lucknow_average_data = []
for ind in range(len(df_lucknow) - window + 1):
    lucknow_average_data.append(np.mean(df_lucknow['PM2.5_StationAverage'][ind:ind+window]))

kolkata_average_data = []
for ind in range(len(df_kolkata) - window + 1):
    kolkata_average_data.append(np.mean(df_kolkata['PM2.5_StationAverage'][ind:ind+window]))


### Plotting Moving Averages

In [9]:
import plotly.graph_objs as go
fig = go.Figure()

In [11]:
len(lucknow_average_data)

544

In [12]:
df_lucknow[:len(lucknow_average_data)].shape

(544, 5)

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_lucknow[:len(lucknow_average_data)]['From'],y=lucknow_average_data,name="Lucknow") )
fig.add_trace(go.Scatter(x=df_kolkata[:len(kolkata_average_data)]['From'],y=kolkata_average_data,name="Kolkata") )
fig.show()

In [14]:
fig.write_html("MovingAvgsLucknowKolkata.html")